<a href="https://colab.research.google.com/github/toraevai/fake_news_classifier/blob/main/%D0%A2%D0%BE%D1%80%D0%B0%D0%B5%D0%B2_%D0%90_%D0%98_%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%961_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
news_df = pd.read_csv('drive/MyDrive/fake_news.csv')

In [ ]:
news_count = pd.DataFrame(news_df['label'].value_counts())
news_count_fig = go.Figure()
news_count_fig.update_layout(
    title='Количество новостей',
    yaxis_title='Количество, шт.',
    xaxis_title='Тип новости'
)
news_count_fig.add_trace(go.Bar(x=news_count.index, y=news_count['count']))

In [ ]:
news_df['label'] = news_df['label'].map(lambda x: 0 if x=='REAL' else 1) # 0 - новость реальна, 1 - новость фейк
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 198.1+ KB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(news_df['text'], news_df['label'], train_size=0.8, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5068,) (5068,)
(1267,) (1267,)


Преобразуем текст в векторы

In [ ]:
vectorizer = TfidfVectorizer(stop_words = 'english')
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
print(X_train_vectorized.shape)
print(X_test_vectorized.shape)

(5068, 61359)
(1267, 61359)


In [ ]:
clf = PassiveAggressiveClassifier()
clf.fit(X_train_vectorized, y_train)

PassiveAggressiveClassifier()

In [ ]:
y_pred = clf.predict(X_test_vectorized)

In [ ]:
score = accuracy_score(y_test, y_pred)
print(f'Accuracy: {round(score * 100, 2)}%')

Accuracy: 93.53%


In [ ]:
model_matrix = confusion_matrix(y_test, y_pred, labels=[0, 1])
model_matrix_df = pd.DataFrame(model_matrix)
model_matrix_df = pd.DataFrame(model_matrix, columns = ['PREDICTED REAL', 'PREDICTED FAKE'], index = ['FACT REAL', 'FAСT FAKE'])
model_matrix_df

,PREDICTED REAL,PREDICTED FAKE
FACT REAL,599,40
FAСT FAKE,42,586


In [ ]:
unique, counts = np.unique(y_pred, return_counts = True)
unique, counts

(array([0, 1]), array([641, 626]))